In [59]:
import requests
import base64
import pandas as pd
import re
import csv
from deep_translator import GoogleTranslator
import time
# 🔹 API Key & Secret
CLIENT_ID = "f234667d9b504925863936ec96d55fed"
CLIENT_SECRET = "626d8b976c8a48ab9f162c0134015560"

# FatSecret data 수집

In [61]:
auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
auth_bytes = auth_string.encode("utf-8")
auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

# 🔹 FatSecret API 토큰 가져오기
def get_access_token():
    url = "https://oauth.fatsecret.com/connect/token"
    headers = {
        "Authorization": f"Basic {auth_base64}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

access_token = get_access_token()

# 🔹 한글 → 영어 번역 캐시
food_translation = {}

# 🔹 한글 → 영어 번역 함수
def translate_food_name(korean_name):
    if korean_name in food_translation:
        return food_translation[korean_name]

    english_name = GoogleTranslator(source="ko", target="en").translate(korean_name)
    food_translation[korean_name] = english_name  # ✅ 변환된 값 저장
    print(f"🌍 번역: {korean_name} → {english_name}")
    return english_name

# 🔹 FatSecret 음식 검색 함수 (자동 번역 포함)
def search_food(query):
    query_eng = translate_food_name(query)  # ✅ 한글 → 영어 자동 변환

    url = "https://platform.fatsecret.com/rest/server.api"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {
        "method": "foods.search",
        "format": "json",
        "search_expression": query_eng
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        result = response.json()
        food_list = result.get("foods", {}).get("food", [])

        if not food_list:  # ✅ 검색 결과가 없는 경우 예외 처리
            print(f"🚨 {query}({query_eng}): 검색 결과 없음!")
            return None

        # ✅ 첫 번째 결과 선택
        if isinstance(food_list, dict):  # 🔸 API가 한 개의 음식만 반환할 경우 dict 형태로 올 수도 있음!
            food_list = [food_list]  # 리스트로 변환

        first_food = food_list[0]
        food_name = first_food.get("food_name", "N/A")
        food_description = first_food.get("food_description", "")

        # ✅ 정규식으로 영양성분 추출
        match = re.search(r'Calories:\s*([\d.]+)kcal\s*\|\s*Fat:\s*([\d.]+)g\s*\|\s*Carbs:\s*([\d.]+)g\s*\|\s*Protein:\s*([\d.]+)g', food_description)

        if match:
            calories, fat, carbs, protein = match.groups()
        else:
            calories = fat = carbs = protein = "N/A"

        return {
            "original_query": query,
            "translated_query": query_eng,
            "food_name": food_name,
            "calories": calories,
            "carbohydrate": carbs,
            "protein": protein,
            "fat": fat
        }
    else:
        print(f"❌ 검색 실패! 상태 코드: {response.status_code}")
        return None

# ✅ CSV 파일에서 음식명 가져오기
df = pd.read_csv("data/merged_food_data.csv") 
food_names = df["식품명"].tolist()  # 🔹 '식품명' 컬럼에서 한글 음식명 가져오기

# ✅ 검색 결과 저장할 리스트
food_results = []

# ✅ 음식명으로 FatSecret 검색 후 결과 저장
for food in food_names:
    print(f"🔍 검색 중: {food}")
    result = search_food(food)
    if result:
        first_food = result  # ✅ 결과 저장
        food_results.append({
            "식품명(한글)": food,
            "식품명(영문)": first_food["translated_query"],
            "검색된 이름": first_food["food_name"],
            "칼로리": first_food["calories"],
            "탄수화물": first_food["carbohydrate"],
            "단백질": first_food["protein"],
            "지방": first_food["fat"]
        })
    else:
        print(f"🚨 {food} 검색 실패!")

    time.sleep(1)  # API 호출 제한 방지 (1초 대기)

# ✅ 결과를 CSV 파일로 저장
output_df = pd.DataFrame(food_results)
output_df.to_csv("data/food_nutrition_data.csv", index=False, encoding="utf-8-sig")

print("✅ 모든 음식 검색 완료! 결과가 'data/food_nutrition_data.csv'에 저장되었습니다.")

🔍 검색 중: 가래떡
🌍 번역: 가래떡 → Sputum
🚨 가래떡(Sputum): 검색 결과 없음!
🚨 가래떡 검색 실패!
🔍 검색 중: 가리비
🌍 번역: 가리비 → scallop
🔍 검색 중: 가지
🌍 번역: 가지 → egg plant
🔍 검색 중: 가지구이
🌍 번역: 가지구이 → Grill
🔍 검색 중: 가츠동
🌍 번역: 가츠동 → Katsu -dong
🔍 검색 중: 간장
🌍 번역: 간장 → soy sauce
🔍 검색 중: 갈비구이
🌍 번역: 갈비구이 → Grilled ribs
🔍 검색 중: 갈비찜
🌍 번역: 갈비찜 → Ribs
🔍 검색 중: 갈비탕
🌍 번역: 갈비탕 → Ribs
🔍 검색 중: 감
🌍 번역: 감 → persimmon
🔍 검색 중: 감귤주스
🌍 번역: 감귤주스 → Citrus juice
🔍 검색 중: 감자구이
🌍 번역: 감자구이 → Potato
🔍 검색 중: 감자그라탕
🌍 번역: 감자그라탕 → Potato
🔍 검색 중: 감자볶음
🌍 번역: 감자볶음 → Potato
🔍 검색 중: 감자샐러드
🌍 번역: 감자샐러드 → Potato salad
🔍 검색 중: 감자스프
🌍 번역: 감자스프 → Potato soup
🔍 검색 중: 감자칩
🌍 번역: 감자칩 → Potato chip
🔍 검색 중: 감자튀김
🌍 번역: 감자튀김 → Fried potato
🔍 검색 중: 거봉포도
🌍 번역: 거봉포도 → Giant grapes
🔍 검색 중: 건자두
🌍 번역: 건자두 → Two
🔍 검색 중: 건크랜베리
🌍 번역: 건크랜베리 → Gun Cranberry
🔍 검색 중: 건포도
🌍 번역: 건포도 → raisin
🔍 검색 중: 게맛살
🌍 번역: 게맛살 → Crab meat
🔍 검색 중: 게장
🌍 번역: 게장 → Crab
🔍 검색 중: 겨자소스
🌍 번역: 겨자소스 → Mustard sauce
🔍 검색 중: 경단
🌍 번역: 경단 → Dumplings
🔍 검색 중: 고갈비
🌍 번역: 고갈비 → Depletion
🚨 고갈비(Depletion): 검색 결과 없음!
🚨 고갈비 검색 실패

# 모델 학습 데이터 전처리

In [ ]:
# 🔹 음식 상세 정보 가져오기
def get_food_details(food_id):
    url = "https://platform.fatsecret.com/rest/server.api"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {
        "method": "food.get",
        "format": "json",
        "food_id": food_id
    }
    
    response = requests.get(url, headers=headers, params=params)
    return response.json()

# 🔹 데이터 정리
food_data = []
for food in diet_foods:
    details = get_food_details(food["food_id"])
    if "food" in details:
        serving = details["food"]["servings"]["serving"]
        food_info = {
            "name": food["food_name"],
            "calories": float(serving["calories"]),
            "carbs": float(serving["carbohydrate"]),
            "protein": float(serving["protein"]),
            "fat": float(serving["fat"])
        }
        food_data.append(food_info)

# 🔹 DataFrame 변환
df = pd.DataFrame(food_data)
print(df)

# XGBoost model 
# 데이터셋 생성

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

# 🔹 예제 데이터 (실제 데이터 확보 후 보강 필요)
df["target"] = [0, 1, 0, 1, 0, 1]  # 0: 다이어트 식단, 1: 근육 증가 식단

# 🔹 입력 데이터(X)와 정답 데이터(y) 분리
X = df[["calories", "carbs", "protein", "fat"]]
y = df["target"]

# 🔹 데이터 분할 (80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 모델 학습

In [ ]:
# 🔹 XGBoost 모델 초기화 및 학습
model = xgb.XGBClassifier(objective="binary:logistic", n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 🔹 정확도 평가
accuracy = model.score(X_test, y_test)
print(f"✅ 모델 정확도: {accuracy * 100:.2f}%")


# 모델 예측

In [ ]:
# 🔹 새로운 음식 데이터 예측 (예: 닭가슴살, 고구마 조합)
new_food = pd.DataFrame([{"calories": 165, "carbs": 20, "protein": 31, "fat": 3.6}])
prediction = model.predict(new_food)

if prediction[0] == 0:
    print("🔥 추천: 다이어트 식단")
else:
    print("💪 추천: 근육 증가 식단")
